## Virtual Env Set and Required Installation

In [22]:
# ! apt install python3.10-venv

In [23]:
# ! python3.10 -m venv rag_prac_env

In [24]:
!source /content/rag_prac_env/bin/activate

### Installation
- openai: LLM model
- langchain: LLM Application Framework
- unstructured: To deal with non-structured data
- chromadb: Vector db
- sentence_transformers: Embedding Model

In [ ]:
# !rag_prac_env/bin/pip install langchain
# !rag_prac_env/bin/pip install openai
# !rag_prac_env/bin/pip install huggingface_hub
# !rag_prac_env/bin/pip install streamlit
# !rag_prac_env/bin/pip install chromadb
# !rag_prac_env/bin/pip install unstructured
# !rag_prac_env/bin/pip install sentence-transformers
# !rag_prac_env/bin/pip install -U langchain-community

## Imports

In [37]:
import sys
import os
sys.path.append('/content/rag_prac_env/lib/python3.10/site-packages')

from google.colab import drive
from google.colab import files
from google.colab import userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dataset Preparation


### Imports
- `from langchain.document_loaders import TextLoader`: To load data
- `from langchain.text_splitter import RecursiveCharacterTextSplitter`: to chunk huge text to into smaller text
- `from langchain.embeddings import SentenceTransformerEmbeddings`: Embedding models to use
- `from langchain.vectorstores import Chroma`: Vectors DB to store after embedding

In [38]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

### Loading & Embedding

In [39]:
# Loading
directory_path = '/content/drive/MyDrive/RAG_DATA'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

given_text = os.path.join(directory_path, 'seongbuk.txt')
documents = TextLoader(given_text).load()

In [40]:
# Chunking
def chunking_dcos(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

chunked_docs = chunking_dcos(documents=documents)

In [41]:
# Embedding Storing
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(chunked_docs, embeddings)

## Retreival & Generation


### Imports

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

os.environ['OPENAI_API_KEY'] = userdata.get('openAI')

### Query, Retreiving & Generating


In [49]:
# LLM Model
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

# Using Q&A Chain to get answer
chain = load_qa_chain(llm, chain_type="stuff", verbose=True)

query = "성북구에서 데이트 코스짜줘"
# Retreiving and Generating
matching_docs = db.similarity_search(query)
answer = chain.run(input_documents=matching_docs, question=query)
answer



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
성북구 나선거구 (정릉1동, 정릉2동, 정릉3동, 정릉4동, 길음1동): 정윤주, 양순임, 이용진
성북구 다선거구 (돈암1동, 종암동): 오중균
성북구 라선거구 (길음2동, 월곡1동, 월곡2동): 이인순, 정해숙
성북구 마선거구 (장위1동, 장위2동): 김경이
성북구 바선거구 (장위3동, 석관동): 이호건
비례대표: 경수현
[5] 성북구 가선거구 (성북동, 삼선동, 동선동, 돈암2동, 안암동, 보문동): 이관우, 정병기
성북구 나선거구 (정릉1동, 정릉2동, 정릉3동, 정릉4동, 길음1동): 임현주, 박영섭
성북구 다선거구 (돈암1동, 종암동): 권영애
성북구 라선거구 (길음2동, 월곡1동, 월곡2동): 이일준
성북구 마선거구 (장위1동, 장위2동): 진선아
성북구 바선거구 (장위3동, 석관동): 정기혁
비례대표: 강수진, 고영옥
[6] 성북구 제2선거구 (정릉1동, 정릉2동, 정릉3동, 정릉4동, 길음1동): 김원중 (초선)
성북구 제4선거구 (장위1동, 장위2동, 장위3동, 석관동): 김태수 (초선)
[7] 성북구 제1선거구 (성북동, 삼선동, 동선동, 돈암2동, 안암동, 보문동): 한신 (초선)
성북구 제3선거구 (돈암제1동, 길음제2동, 종암동, 월곡제1동, 월곡제2동): 강동길 (재선)
[8] 노원구, 중랑구와 접하긴 하지만 실제로 성북구의 중심가와는 멀리 떨어져 있으며, 성북구의 미아리고개 너머 지역하고도 인접해 있다고 보기는 어렵고 석관

'죄송합니다. 제가 성북구에 관한 데이트 코스에 대해 알고 있지는 않습니다. 다른 도움이 필요하시다면 더 알려드릴 수 있는 것이 있다면 알려주세요!'